# [External Runtime] Fairness Indicators Example Colab

**Overview**

In this activity, you'll use Fairness Indicators to explore the Civil Comments dataset. Fairness Indicators is a suite of tools built on top of [TensorFlow Model Analysis](https://www.tensorflow.org/tfx/model_analysis/get_started) that enable regular evaluation of fairness metrics in product pipelines.

**About the Dataset**

In this exercise, you'll work with the [Civil Comments dataset](https://www.kaggle.com/c/jigsaw-unintended-bias-in-toxicity-classification), approximately 2 million public comments made public by the [Civil Comments platform](https://medium.com/@aja_15265/saying-goodbye-to-civil-comments-41859d3a2b1d) in 2017 for ongoing research. This effort was sponsored by Jigsaw, who have hosted competitions on Kaggle to help classify toxic comments as well as minimize unintended model bias.

Each individual text comment in the dataset has a toxicity label, with the label being 1 if the comment is toxic and 0 if the comment is non-toxic. Within the data, a subset of comments are labeled with a variety of identity attributes, including categories for gender, sexual orientation, religion, and race or ethnicity.

# Importing

Run the following code to install the fairness_indicators library. This package contains the tools we'll be using in this exercise.

In [0]:
# TODO: !pip install fairness_indicators
%tensorflow_version 1.x
!pip install tensorflow_model_analysis
!pip install tensorflow_data_validation
!pip install --upgrade witwidget

**Restart the runtime as needed.** Run the following code to import the necessary dependencies for the libraries.

In [0]:
import os
import tempfile
import apache_beam as beam
import numpy as np
import pandas as pd
from datetime import datetime

import tensorflow as tf
tf.enable_v2_behavior()

import tensorflow_hub as hub
import tensorflow_model_analysis as tfma
import tensorflow_data_validation as tfdv
from tensorflow_model_analysis.addons.fairness.post_export_metrics import fairness_indicators
from tensorflow_model_analysis.addons.fairness.view import widget_view

from witwidget.notebook.visualization import WitConfigBuilder
from witwidget.notebook.visualization import WitWidget

tf.get_logger().propagate = False

In [0]:
print('TensorFlow version: {}'.format(tf.__version__))

# Download the Data

In this exercise, you'll work with the Civil Comments dataset, approximately 2 million public comments made public by the Civil Comments platform in 2017.

We've hosted the dataset on Google Cloud Platform for convenience. Run the following code to download the data from GCP:

In [0]:
train_tf_file = tf.keras.utils.get_file('train.tf', 'https://storage.googleapis.com/civil_comments_dataset/train.tfrecord')
validate_tf_file = tf.keras.utils.get_file('validate.tf', 'https://storage.googleapis.com/civil_comments_dataset/validate.tfrecord')

# Defining Constants

Here, we define the feature map that will be used to parse the data. Each example will have a label, comment text, and identity features `sexual orientation`, `gender`, `religion`, `race`, and `disability` that are associated with the text. See [this page](https://www.kaggle.com/c/jigsaw-unintended-bias-in-toxicity-classification/data) to learn more about the data schema. 

In [0]:
BASE_DIR = tempfile.gettempdir()

TEXT_FEATURE = 'comment_text'
LABEL = 'toxicity'
FEATURE_MAP = {
    # Label:
    LABEL: tf.io.FixedLenFeature([], tf.float32),
    # Text:
    TEXT_FEATURE:  tf.io.FixedLenFeature([], tf.string),

    # Identities:
    'sexual_orientation':tf.io.VarLenFeature(tf.string),
    'gender':tf.io.VarLenFeature(tf.string),
    'religion':tf.io.VarLenFeature(tf.string),
    'race':tf.io.VarLenFeature(tf.string),
    'disability':tf.io.VarLenFeature(tf.string),
}

# Train the Model

First, set up the input function to feed data into the model:

In [0]:
def train_input_fn():
  def parse_function(serialized):
    parsed_example = tf.io.parse_single_example(
        serialized=serialized, features=FEATURE_MAP)
    # Adds a weight column to deal with unbalanced classes.
    parsed_example['weight'] = tf.add(parsed_example[LABEL], 0.1)
    return (parsed_example,
            parsed_example[LABEL])
  train_dataset = tf.data.TFRecordDataset(
      filenames=[train_tf_file]).map(parse_function).batch(512)
  return train_dataset

Next, create a deep neural network model, and train it on the data:

In [0]:
model_dir = os.path.join(BASE_DIR, 'train', datetime.now().strftime(
    "%Y%m%d-%H%M%S"))

embedded_text_feature_column = hub.text_embedding_column(
    key=TEXT_FEATURE,
    module_spec='https://tfhub.dev/google/nnlm-en-dim128/1')

classifier = tf.estimator.DNNClassifier(
    hidden_units=[500, 100],
    weight_column='weight',
    feature_columns=[embedded_text_feature_column],
    n_classes=2,
    optimizer=tf.train.AdagradOptimizer(learning_rate=0.003),
    model_dir=model_dir)

classifier.train(input_fn=train_input_fn, steps=1000)

# Run TensorFlow Model Analysis with Fairness Indicators

**TODO:** Description of TFMA and the added value of Fairness Indicators

## Export Saved Model

In [0]:
def eval_input_receiver_fn():
  serialized_tf_example = tf.compat.v1.placeholder(
      dtype=tf.string, shape=[None], name='input_example_placeholder')

  # This *must* be a dictionary containing a single key 'examples', which
  # points to the input placeholder.
  receiver_tensors = {'examples': serialized_tf_example}

  features = tf.io.parse_example(serialized_tf_example, FEATURE_MAP)
  features['weight'] = tf.ones_like(features[LABEL])

  return tfma.export.EvalInputReceiver(
    features=features,
    receiver_tensors=receiver_tensors,
    labels=features[LABEL])

tfma_export_dir = tfma.export.export_eval_savedmodel(
  estimator=classifier,
  export_dir_base=os.path.join(BASE_DIR, 'tfma_eval_model'),
  eval_input_receiver_fn=eval_input_receiver_fn)

## Compute Fairness Metrics

Render the Fairness Indicators widget with the exported evaluation results:

In [0]:
tfma_eval_result_path = os.path.join(BASE_DIR, 'tfma_eval_result')

# Define slices that you want the evaluation to run on.
slice_spec = [
    tfma.slicer.SingleSliceSpec(), # Overall slice
    tfma.slicer.SingleSliceSpec(columns=['sexual_orientation']),
    # You can uncomment the following lines for experiments on other identities.
    # tfma.slicer.SingleSliceSpec(columns=['gender'],),
    # tfma.slicer.SingleSliceSpec(columns=['religion']),
    # tfma.slicer.SingleSliceSpec(columns=['race']),
    # tfma.slicer.SingleSliceSpec(columns=['disability']),
]

# Add the fairness metrics.
add_metrics_callbacks = [
  tfma.post_export_metrics.fairness_indicators(
      thresholds=[0.1, 0.3, 0.5, 0.7, 0.9],
      labels_key=LABEL
      )
]

eval_shared_model = tfma.default_eval_shared_model(
    eval_saved_model_path=tfma_export_dir,
    add_metrics_callbacks=add_metrics_callbacks)

validate_dataset = tf.data.TFRecordDataset(filenames=[validate_tf_file])

# Run the fairness evaluation.
with beam.Pipeline() as pipeline:
  _ = (
      pipeline
      | beam.Create([v.numpy() for v in validate_dataset])
      | 'ExtractEvaluateAndWriteResults' >>
       tfma.ExtractEvaluateAndWriteResults(
                 eval_shared_model=eval_shared_model,
                 slice_spec=slice_spec,
                 compute_confidence_intervals=True,
                 output_path=tfma_eval_result_path)
  )

eval_result = tfma.load_eval_result(output_path=tfma_eval_result_path)

In [0]:
widget_view.render_fairness_indicator(eval_result)

# Run What-if Tool

In this section, you'll use the [What-If Tool's ](https://pair-code.github.io/what-if-tool/)interactive visual interface to explore and manipulate data at a micro-level.

You can use the **Binning**, **Color By**, **Label by**, and **Scatter** dropdowns at the top of the What-If widget to create a visualization that groups examples by sexual orientation, and displays both how each example was categorized (`inference_label`) by the model and its actual ground-truth label (`toxicity`).


In [0]:
num_datapoints = 1000
def wit_dataset(file, num_datapoints):
  dataset = tf.data.TFRecordDataset(
      filenames=[train_tf_file]).take(num_datapoints)
  return [tf.train.Example.FromString(d.numpy()) for d in dataset]

data = wit_dataset(train_tf_file, num_datapoints)
config_builder = WitConfigBuilder(data).set_estimator_and_feature_spec(
    classifier, FEATURE_MAP).set_label_vocab(['non-toxicity', LABEL]).set_target_feature(LABEL)
wit = WitWidget(config_builder)
wit

In the Fairness Indicators widget above, you can click on a particular a slice to further explore it in the What-If Tool. The data in that slice will appear in the What-If Tool widget for further inspection.

# Run TensorFlow Data Validation

[TensorFlow Data Validation](https://www.tensorflow.org/tfx/guide/tfdv) is another tool you can use to analyze your data. You can use it to find potential problems in your data, such as missing values and data imbalances, that can lead to Fairness disparities.

In [0]:
stats = tfdv.generate_statistics_from_tfrecord(data_location=train_tf_file)
tfdv.visualize_statistics(stats)